In [1]:
import pandas as pd
from scipy.io import arff


In [2]:
# data = arff.loadarff('dataset_37_diabetes.arff')
# train= pd.DataFrame(data[0])
# train.head()

In [3]:
# train.to_csv('Diabetes.csv',index=False)

In [4]:
new = pd.read_csv("Diabetes.csv")

In [5]:
new

,preg,plas,pres,skin,insu,mass,pedi,age,class
0,6.0,148.0,72.0,35.0,0.0,33.6,0.627,50.0,b'tested_positive'
1,1.0,85.0,66.0,29.0,0.0,26.6,0.351,31.0,b'tested_negative'
2,8.0,183.0,64.0,0.0,0.0,23.3,0.672,32.0,b'tested_positive'
3,1.0,89.0,66.0,23.0,94.0,28.1,0.167,21.0,b'tested_negative'
4,0.0,137.0,40.0,35.0,168.0,43.1,2.288,33.0,b'tested_positive'
...,...,...,...,...,...,...,...,...,...
763,10.0,101.0,76.0,48.0,180.0,32.9,0.171,63.0,b'tested_negative'
764,2.0,122.0,70.0,27.0,0.0,36.8,0.340,27.0,b'tested_negative'
765,5.0,121.0,72.0,23.0,112.0,26.2,0.245,30.0,b'tested_negative'
766,1.0,126.0,60.0,0.0,0.0,30.1,0.349,47.0,b'tested_positive'


In [6]:
new.isnull().sum()

preg     0
plas     0
pres     0
skin     0
insu     0
mass     0
pedi     0
age      0
class    0
dtype: int64

In [7]:
new['class'].value_counts()

b'tested_negative'    500
b'tested_positive'    268
Name: class, dtype: int64

In [6]:
import h2o
from h2o.automl import H2OAutoML

In [7]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,5 days 3 hours 28 mins
H2O_cluster_timezone:,Europe/London
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.42.0.1
H2O_cluster_version_age:,"28 days, 6 hours and 11 minutes"
H2O_cluster_name:,H2O_from_python_lakshmiph_d0splj
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,452.7 Mb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [8]:
# convert pandas DataFrame into H2O Frame
new_df = h2o.H2OFrame(new)
# Describe  the train h20Frame
new_df.describe()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


Rows:768
Cols:9

,preg,plas,pres,skin,insu,mass,pedi,age,class
type,int,int,int,int,int,real,real,int,enum
mins,0.0,0.0,0.0,0.0,0.0,0.0,0.078,21.0,
mean,3.845052083333336,120.89453125,69.10546874999994,20.536458333333357,79.79947916666666,31.99257812500003,0.4718763020833334,33.240885416666615,
maxs,17.0,199.0,122.0,99.0,846.0,67.1,2.42,81.0,
sigma,3.36957806269887,31.972618195136224,19.355807170644777,15.952217567727642,115.24400235133803,7.884160320375441,0.331328595012775,11.760231540678689,
zeros,111,5,35,227,374,11,0,0,
missing,0,0,0,0,0,0,0,0,0
0,6.0,148.0,72.0,35.0,0.0,33.6,0.627,50.0,b'tested_positive'
1,1.0,85.0,66.0,29.0,0.0,26.6,0.351,31.0,b'tested_negative'
2,8.0,183.0,64.0,0.0,0.0,23.3,0.672,32.0,b'tested_positive'


In [9]:
x = new_df.columns
y = 'class'
# remove label classvariable from feature variable
x.remove(y)

In [10]:
train, test = new_df.split_frame(ratios = [.8], seed = 1234)

In [11]:
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [12]:
aml = H2OAutoML(max_runtime_secs=350,
                exclude_algos =['DeepLearning'],
                seed = 1,
                balance_classes = False,
)
# train model and record time % time
aml.train(x = x, y = y, training_frame = train, validation_frame=test)

AutoML progress: |
00:00:08.103: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.
00:00:08.104: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),4/4
# GBM base models (used / total),1/1
# GLM base models (used / total),1/1
# DRF base models (used / total),2/2
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None
Custom metalearner hyperparameters,None


In [13]:
best_model = aml.get_best_model()
print(best_model)

Model Details
H2OStackedEnsembleEstimator : Stacked Ensemble
Model Key: StackedEnsemble_BestOfFamily_7_AutoML_12_20230719_00008


Model Summary for Stacked Ensemble: 
key                                   value
------------------------------------  ----------------
Stacking strategy                     cross_validation
Number of base models (used / total)  4/4
# GBM base models (used / total)      1/1
# GLM base models (used / total)      1/1
# DRF base models (used / total)      2/2
Metalearner algorithm                 GLM
Metalearner fold assignment scheme    Random
Metalearner nfolds                    5
Metalearner fold_column
Custom metalearner hyperparameters    None

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.12543895694289772
RMSE: 0.3541736254196488
LogLoss: 0.39697665847777563
AUC: 0.8976283573057766
AUCPR: 0.8293022674807925
Gini: 0.7952567146115532
Null degrees of freedom: 612
Residual degrees of freedom: 608
Null deviance: 796.759904707